In [ ]:
import pandas as pd

# Step 1: Read the car rental data CSV
car_rental_df = pd.read_excel('Cars_sales_data.xlsx')

def custom_date_parser(date_string):
    # Assuming the original date format is "DD/MM/YYYY"
    return pd.to_datetime(date_string, format='%d/%m/%Y')

# Function to process the newest penalty form entry
def process_newest_penalty_form():
    # Read the penalty form data CSV to get the newest entry
    penalty_form_df = pd.read_excel('penalty_data.xlsx', parse_dates=['Date'], date_parser=custom_date_parser)
    newest_entry = penalty_form_df.tail(1)  # Get the last row, which is the newest entry
    plate_number = newest_entry['Plate Number'].iloc[0]
    penalty_date = newest_entry['Date'].iloc[0].strftime('%d/%m/%Y')
    penalty_time = newest_entry['Time Penalty'].iloc[0]

    # Compare with car rental data
    matched_data = car_rental_df[
        (car_rental_df['Plate Number'] == plate_number) &
        (car_rental_df['Date'] == penalty_date)
    ]

    if not matched_data.empty:
        # Check if both Pick up time and Drop off time are not null
        if 'Pick up time' in car_rental_df.columns and 'Drop off time' in car_rental_df.columns:
            matched_data = matched_data[
                ((matched_data['Pick up time'].notnull()) & (matched_data['Drop off time'].notnull()) &
                (matched_data['Pick up time'] <= penalty_time) & (penalty_time <= matched_data['Drop off time']))
                |
                ((matched_data['Pick up time'].notnull()) & (matched_data['Drop off time'].isnull()) &
                (matched_data['Pick up time'] <= penalty_time))
                |
                ((matched_data['Pick up time'].isnull()) & (matched_data['Drop off time'].notnull()) &
                (penalty_time <= matched_data['Drop off time']))
                |
                ((matched_data['Pick up time'].isnull()) & (matched_data['Drop off time'].isnull()))
            ]


        if not matched_data.empty:
            # Perform inner join
            joined_data = pd.merge(matched_data, newest_entry, on='Plate Number')

            joined_data.rename(columns={'Date_x' : 'Date' , 'Date_y' : 'Date Penalty'}, inplace=True)
            # Append joined data to a third CSV
            with open('joined_data.csv', 'a') as f:
                joined_data.to_csv(f, mode='a', header=f.tell()==0, index=False)
            print("New penalty form processed and appended successfully.")
        else:
            print("No matching rental data found for the penalty form.")
    else:
        print("No matching rental data found for the penalty form.")

# Process the newest penalty form
process_newest_penalty_form()

New penalty form processed and appended successfully.


<ipython-input-12-dadfef8ee69a>:13: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  penalty_form_df = pd.read_excel('penalty_data.xlsx', parse_dates=['Date'], date_parser=custom_date_parser)
